In [1]:
import gensim
import pandas as pd
import jieba
import re
import math
import numpy as np
import random

In [5]:
!pip install stanfordcorenlp -i http://pypi.mirrors.ustc.edu.cn/simple/ --trusted-host pypi.mirrors.ustc.edu.cn

Looking in indexes: http://pypi.mirrors.ustc.edu.cn/simple/


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import stanfordcorenlp

In [3]:
data = pd.read_csv('./sqlResult_1558435.csv', encoding='gb18030')
data = data.fillna('')
def cut(string):
    return ' '.join(jieba.cut(string))

In [11]:
cut('床前明月光')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LGP\AppData\Local\Temp\jieba.cache
Loading model cost 1.720 seconds.
Prefix dict has been built successfully.


'床前 明月光'

In [4]:
data['content'][10]

'\u3000\u3000证券时报网（www.stcn.com）06月23日讯\r\n\u3000\u3000据上证报道，6月初以来，创业板指数明显强于上证综指，一时间关于蓝筹行情将向成长股行情切换的预期再起。但多位私募人士表示，蓝筹股与成长股这两年并非绝对的此消彼长。本轮蓝筹股上涨行情的主要推动力就是业绩成长，而业绩与估值匹配才是王道。上述私募人士依然看好成长性突出的蓝筹股。\r\n'

In [5]:
def token(string):
    return re.findall(r'[\w|\d]+',string)

In [6]:
token(data['content'][10])

['证券时报网',
 'www',
 'stcn',
 'com',
 '06月23日讯',
 '据上证报道',
 '6月初以来',
 '创业板指数明显强于上证综指',
 '一时间关于蓝筹行情将向成长股行情切换的预期再起',
 '但多位私募人士表示',
 '蓝筹股与成长股这两年并非绝对的此消彼长',
 '本轮蓝筹股上涨行情的主要推动力就是业绩成长',
 '而业绩与估值匹配才是王道',
 '上述私募人士依然看好成长性突出的蓝筹股']

In [7]:
content = data['content'].tolist()

In [8]:
token(content[10])

['证券时报网',
 'www',
 'stcn',
 'com',
 '06月23日讯',
 '据上证报道',
 '6月初以来',
 '创业板指数明显强于上证综指',
 '一时间关于蓝筹行情将向成长股行情切换的预期再起',
 '但多位私募人士表示',
 '蓝筹股与成长股这两年并非绝对的此消彼长',
 '本轮蓝筹股上涨行情的主要推动力就是业绩成长',
 '而业绩与估值匹配才是王道',
 '上述私募人士依然看好成长性突出的蓝筹股']

In [9]:
content_new = [token(i) for i in content]
content_new = [''.join(j) for j in content_new]
content_new = [cut(n) for n in content_new]
content_new[0]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LGP\AppData\Local\Temp\jieba.cache
Loading model cost 1.586 seconds.
Prefix dict has been built successfully.


'此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息'

In [10]:
def doc_freq(word):
    return sum(1 for m in content_new if word in m)
def IDF(word):
    return math.log10(len(content_new)/doc_freq(word))

In [11]:
IDF("小米")

2.9523613237924735

In [12]:
def tf(word,doc):
    words = doc.split()
    return sum(1 for i in words if i==word)

In [13]:
def tf_idf(word,document):
    return tf(word,document)*IDF(word)

In [14]:
def get_key_word(doc):
    words = set(doc.split())
    tfidf = [(w, tf_idf(w,doc)) for w in words]
    tfidf = sorted(tfidf, key=lambda x: x[1],reverse=True)
    return tfidf

In [28]:
get_key_word(content_new[0])

[('MIUI9', 9.904722647584947),
 ('机型', 5.854110917055407),
 ('体验版', 4.9523613237924735),
 ('稳定版', 4.9523613237924735),
 ('暂不受', 4.9523613237924735),
 ('MIUI8', 4.9523613237924735),
 ('系统优化', 4.9523613237924735),
 ('精力', 4.865066660033509),
 ('内测', 4.17421007340883),
 ('官方消息', 3.9523613237924735),
 ('距今已有', 3.6301420290585544),
 ('更新换代', 3.433847383914586),
 ('用到', 3.339577467072738),
 ('确切', 3.139447967149618),
 ('有余', 3.0128420711738553),
 ('小米', 2.9523613237924735),
 ('猜测', 2.6828483795745575),
 ('本周', 2.310887213288374),
 ('其余', 2.2953054709353697),
 ('工程师', 2.2595144045152438),
 ('更新', 2.2143739974590426),
 ('暂停', 2.188933330229536),
 ('发布', 2.15877938610806),
 ('之中', 2.129539678489369),
 ('当然', 2.040139267260058),
 ('等待', 2.031716322385686),
 ('日起', 1.8146907865557185),
 ('有人', 1.7490564076539907),
 ('研发', 1.7322532357524185),
 ('手机', 1.7180971994136842),
 ('确保', 1.6071837072497697),
 ('含', 1.5902560044987006),
 ('集中', 1.558260021752429),
 ('时候', 1.5056688574209465),
 ('版', 1.4878

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 把原始文本转化为tf-idf的特征矩阵，从而为后续的文本相似度计算，主题模型(如LSI)，文本搜索排序等一系列应用奠定基础
vectorizer = TfidfVectorizer(max_features=5000)
sample_num = 20000
sub_samples = content_new[:sample_num]
X = vectorizer.fit_transform(sub_samples)
len(vectorizer.vocabulary_)

5000

In [16]:
print(X)

  (0, 3179)	0.14139588907091524
  (0, 3036)	0.21320986621640167
  (0, 15)	0.13380933164904654
  (0, 2887)	0.19145894254076248
  (0, 2079)	0.26705675648090177
  (0, 2527)	0.17840644790266266
  (0, 20)	0.13827767524810422
  (0, 983)	0.21896156389178495
  (0, 2948)	0.21320986621640167
  (0, 2962)	0.21853593990484893
  (0, 1383)	0.2615613738186619
  (0, 2279)	0.15942234345286213
  (0, 2340)	0.1288801474529989
  (0, 3717)	0.1659120353218626
  (0, 2150)	0.24243054168052397
  (0, 1437)	0.1125420294783423
  (0, 4847)	0.16317289770821933
  (0, 930)	0.14598188404982876
  (0, 4613)	0.0979256686508875
  (0, 2143)	0.1143392179693401
  (0, 2993)	0.17199589422178171
  (0, 504)	0.11848681178686539
  (0, 3705)	0.1731432408321265
  (0, 516)	0.2152911850808729
  (0, 1344)	0.13783752305804667
  :	:
  (19999, 3443)	0.07372030912603598
  (19999, 4756)	0.13197699714527059
  (19999, 25)	0.1357343186901138
  (19999, 432)	0.08132405489694577
  (19999, 46)	0.06718516609256403
  (19999, 2328)	0.0933624738086428
 

In [17]:
np.where(X[1].toarray()) 
# 当参数中只给出条件时，则返回非零元素的索引：返回的第一个array表示行坐标，第二个array表示纵坐标，两者一一对应

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([  61,  159,  210,  219,  264,  318,  534,  609,  665,  679,  688,
         735,  772, 1431, 1437, 1564, 1609, 1974, 2079, 2162, 2206, 2283,
        2315, 2331, 2336, 2368, 2396, 2559, 2598, 2630, 2846, 2931, 2960,
        3032, 3123, 3412, 3527, 3572, 3642, 3798, 4028, 4046, 4047, 4109,
        4258, 4304, 4437, 4603, 4639, 4667, 4894, 4930], dtype=int64))

In [18]:
document_1,document_2 = random.randint(0,1000),random.randint(0,1000)

In [22]:
content_new[document_1]
vector_of_document_1 = X[document_1].toarray()[0]
content_new[document_2]
vector_of_document_2 = X[document_2].toarray()[0]
print(vector_of_document_1)
print(vector_of_document_2)

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


In [23]:
from scipy.spatial.distance import cosine
def distance(v1,v2): 
    return cosine(v1,v2)
distance(vector_of_document_1,vector_of_document_2)

0.9940556085839487

In [34]:
from gensim import corpora, models
import jieba.posseg as jp # 词性标注

In [25]:
data = pd.read_csv('./sqlResult_1558435.csv', encoding='gb18030')

In [33]:
flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')

In [28]:
news = data["content"][:10]

In [35]:
stop_word = []
with open('./百度停用词表.txt',encoding = 'utf-8') as f:
    for words in f.readlines():
        stop_word.append(words.strip())
# print(stop_word)

In [36]:
word_ls = []
for text in news:
    words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_word] # w.flag为w的词性
    word_ls.append(words)

In [37]:
#建立语料库
dictionary = corpora.Dictionary(word_ls) # 可以理解为python中的字典对象, 其Key是字典中的词，其Val是词对应的唯一数值型ID

In [41]:
corpus = [dictionary.doc2bow(words) for words in word_ls]
print(corpus)

[[(0, 3), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1)], [(8, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 3), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 3), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 3), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1)], [(2, 1), (12, 2), (79, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 3), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 4), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), 

In [42]:
lda = models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary,num_topics=5)
for topic in lda.print_topics(num_words=10):
    print(topic)

(0, '0.009*"市场" + 0.008*"银行" + 0.006*"指数" + 0.006*"女子" + 0.006*"板块" + 0.006*"资金" + 0.005*"A股" + 0.005*"MSCI" + 0.005*"刘青" + 0.004*"金融股"')
(1, '0.013*"市场" + 0.008*"个股" + 0.006*"资金" + 0.006*"手机" + 0.006*"指数" + 0.005*"上涨" + 0.005*"A股" + 0.005*"会" + 0.005*"平台" + 0.004*"板块"')
(2, '0.018*"女子" + 0.009*"刘青" + 0.006*"时" + 0.005*"会" + 0.005*"银行" + 0.004*"发现" + 0.004*"手机" + 0.004*"信息" + 0.004*"市场" + 0.004*"指数"')
(3, '0.009*"市场" + 0.007*"指数" + 0.006*"个股" + 0.005*"手机" + 0.005*"会" + 0.005*"网络" + 0.005*"中国" + 0.004*"板块" + 0.004*"信息" + 0.004*"金融"')
(4, '0.010*"女子" + 0.010*"市场" + 0.007*"资金" + 0.006*"会" + 0.006*"个股" + 0.006*"板块" + 0.006*"指数" + 0.005*"信息" + 0.005*"银行" + 0.004*"刘青"')
